# Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
# 1) Camera calibration using chessboard images

In [ ]:
# Importing some useful packages
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import pickle
%matplotlib inline

In [ ]:
# Helper functions
# Show 2 images on the same line for easy comparison
def show2img(img1, img2, title1, title2):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img1)
    ax1.set_title(title1, fontsize=50)
    ax2.imshow(img2)
    ax2.set_title(title2, fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Show 2 images (the second one with gray scale) on the same line for easy comparison
def show2img_gray(img1, img2, title1, title2):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img1)
    ax1.set_title(title1, fontsize=50)
    ax2.imshow(img2, cmap='gray')
    ax2.set_title(title2, fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Show 3 images on the same line for easy comparison
def show3img(img1, img2, img3, title1, title2, title3):
    f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 6))
    f.tight_layout()
    ax1.imshow(img1)
    ax1.set_title(title1, fontsize=50)
    ax2.imshow(img2)
    ax2.set_title(title2, fontsize=50)
    ax3.imshow(img3)
    ax3.set_title(title3, fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
nx = 9 # the number of inside corners in x
ny = 6 # the number of inside corners in y

objp = np.zeros((ny*nx,3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

In [ ]:
# Make a list of calibration images
images = os.listdir("camera_cal/")

# Step through the list and search for chessboard corners
for fname in images:
    # Read image
    img = cv2.imread('camera_cal/'+fname)
    # Create grayscale image
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)

    if ret == True:
        # If found, add object points, image points
        objpoints.append(objp)
        imgpoints.append(corners)
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
        write_name = 'output_images/corners_'+fname
        cv2.imwrite(write_name, img)
    else:
        # If not, return the filename
        print("This image can't be processed : "+fname)

In [ ]:
# Show an image (calibration2.jpg) & its chessboard corners
img1 = cv2.imread('camera_cal/calibration2.jpg')
img2 = cv2.imread('output_images/corners_calibration2.jpg')
show2img(img1, img2, 'Calibration image', 'Image with chessboard corners')

In [ ]:
# Apply camera calibration to get the matrix (mtx) and distortion coefficients (dist)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Print out the value of mxt & dist
print("Here is the matrix (mtx):")
print(mtx)
print("Here is the distortion coefficients (dist):")
print(dist)

In [ ]:
# Save the camera calibration result for later use
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump(dist_pickle, open("calibrated_result.p","wb"))

# 2) Apply a distortion correction to raw images

In [ ]:
# Read the matrix (mtx) and coefficients (dist) from calibrated result
# Use the saved data to avoid running the calibration from the beginning when restarting kernel, etc
dist_pickle = pickle.load(open("calibrated_result.p", "rb"))
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

In [ ]:
# Undistort each chessboard image using the calculated matrix (mtx) & coefficients (dist)
for fname in images:
    # Read image
    img = cv2.imread('camera_cal/'+fname)
    # Create undistorted image & save to file in "output_images" folder
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    write_name = 'output_images/undistorted_'+fname
    cv2.imwrite(write_name, undist)

In [ ]:
# Show 3 images: original, with chessboard corners, undistorted
img1 = cv2.imread('camera_cal/calibration3.jpg')
img2 = cv2.imread('output_images/corners_calibration3.jpg')
img3 = cv2.imread('output_images/undistorted_calibration3.jpg')
show3img(img1, img2, img3, 'Original', 'With corners', 'Undistorted')

In [ ]:
# Generate undistorted image and show with its raw image
raw_original = mpimg.imread('test_images/test1.jpg')
raw_undist = cv2.undistort(raw_original, mtx, dist, None, mtx)
show2img(raw_original, raw_undist, 'Original image', 'Undistorted image')

# 3) Use color transforms, gradients, etc., to create a thresholded binary image.
### Implement some helper functions

In [ ]:
# A function that takes an image, gradient orientation,
# and threshold min / max values.
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold to generate binary image
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1 
    return grad_binary

# A function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    return mag_binary

# A function to return the direction of the gradient
# for a given sobel kernel size and threshold values
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    dir_binary =  np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    return dir_binary

# A function to threshold an image for a given range of color
def color_thresh(img, color_thresh):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    color_binary = np.zeros_like(gray)
    color_binary[(gray > color_thresh[0]) & (gray <= color_thresh[1])] = 1
    return color_binary

# A function that thresholds the S-channel of HLS
def hls_select(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    hls_binary = np.zeros_like(s_channel)
    hls_binary[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return hls_binary

### Apply to test image to create thresholded binary images

In [ ]:
# Read the image to be tested
test_img = mpimg.imread('test_images/test1.jpg')
# Undistort the test image
test_img_undist = cv2.undistort(test_img, mtx, dist, None, mtx)

In [ ]:
# Call the function to apply gradient threshold (on x direction)
gradx = abs_sobel_thresh(test_img_undist, orient='x', thresh=(20,100))
# Show 2 images (original & binary)
show2img_gray(test_img_undist, gradx, "Original Image", "Gradient Threshold (x axis)")

In [ ]:
# Call the function to apply gradient threshold (on y direction)
grady = abs_sobel_thresh(test_img_undist, orient='y', thresh=(20,100))
# Show 2 images (original & binary)
show2img_gray(test_img_undist, grady, "Original Image", "Gradient Threshold(y axis)")

In [ ]:
# Call the function to apply gradient magnitude threshold
mag_binary = mag_thresh(test_img_undist, sobel_kernel=3, mag_thresh=(30, 100))
# Show 2 images (original & binary)
show2img_gray(test_img_undist, mag_binary, "Original Image", "Magnitude Threshold")

In [ ]:
# Call the function to apply gradient direction threshold
dir_binary = dir_threshold(test_img_undist, sobel_kernel=15, thresh=(0.7, 1.3))
# Show 2 images (original & binary)
show2img_gray(test_img_undist, dir_binary, "Original Image", "Direction Threshold")

In [ ]:
# Binary image with combined thresholds
combined = np.zeros_like(dir_binary)
combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
# Show 2 images (original & binary)
show2img_gray(test_img_undist, combined, "Original Image", "Combined thresholds")

In [ ]:
# Binary image with color threshold
color_binary = color_thresh(test_img_undist, color_thresh=(200, 255))
# Show 2 images (original & binary)
show2img_gray(test_img_undist, color_binary, "Original Image", "Color Threshold")

In [ ]:
# Binary image with color threshold
hls_binary = hls_select(test_img_undist, thresh=(180, 255))
# Show 2 images (original & binary)
show2img_gray(test_img_undist, hls_binary, "Original Image", "S-Channel (HLS) Threshold")

# 4) Apply a perspective transform to rectify binary image ("birds-eye view")

In [ ]:
# A function to perform perspective transform to image from given points (src & dst)
def warped(img, src, dst):
    # Get image size
    img_size = (img.shape[1], img.shape[0])
    # Calculate transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    # Apply perpective transform with same size as input image
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_NEAREST)
    return warped, M

In [ ]:
# Choose reference points for the transform (src & dst)
# Note: currently picking up from writeup template. To be optimized later.
img_size = (test_img.shape[1], test_img.shape[0])

src = np.float32(
    [[(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
    [((img_size[0] / 6) - 10), img_size[1]],
    [(img_size[0] * 5 / 6) + 60, img_size[1]],
    [(img_size[0] / 2 + 55), img_size[1] / 2 + 100]])
dst = np.float32(
    [[(img_size[0] / 4), 0],
    [(img_size[0] / 4), img_size[1]],
    [(img_size[0] * 3 / 4), img_size[1]],
    [(img_size[0] * 3 / 4), 0]])

In [ ]:
# Create a warped image by perspective transform
test_img_warped, M_perspective = warped(test_img_undist, src, dst)
# Show 2 images
show2img(test_img_undist, test_img_warped, "Undistorted Image", "Warped Image")

In [ ]:
# Create a warped binary image by perspective transform
color_img_warped, M_perspective = warped(color_binary, src, dst)
# Show 2 images
show2img_gray(test_img_warped, color_img_warped, "Warped Image", "Warped Binary Image")

In [ ]:
# Read the test image with straight line
straight_img = mpimg.imread('test_images/straight_lines1.jpg')
# Undistort the test image
straight_img_undist = cv2.undistort(straight_img, mtx, dist, None, mtx)
# Create a warped image by perspective transform 
straight_img_warped, M_perspective = warped(straight_img_undist, src, dst)
# Show 2 images
show2img(straight_img_undist, straight_img_warped, "Undistorted Image", "Warped Image")

# 5) Detect lane pixels and fit to find the lane boundary

In [ ]:
# A function to create histogram of given image
def hist(img):
    # Grab only the bottom half of the image
    # Lane lines are likely to be mostly vertical nearest to the car
    bottom_half = img[img.shape[0]//2:,:]

    # Sum across image pixels vertically - make sure to set an `axis`
    # i.e. the highest areas of vertical lines should be larger values
    histogram = np.sum(bottom_half, axis=0)
    
    return histogram

In [ ]:
# Read the binary image to be called for histogram creation
# Specify here a binary image which is thresholded & transformed above
# Temporarily use "color_img_warped" as an example
binaryimage = color_img_warped

# Create histogram of image binary activations
histogram = hist(binaryimage)

# Visualize the resulting histogram
plt.plot(histogram)

In [ ]:
# A function to find lane pixels from binary warped image
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

# A function to fit a polynomial from detected lane pixels
def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')

    return out_img

In [ ]:
# Apply the process (to find lane pixels & fit polynomial) to binary image
out_img = fit_polynomial(color_img_warped)
# Show the output image
plt.imshow(out_img)

# 6) Determine the curvature of the lane and vehicle position with respect to center.

### Sliding Window and Search from Prior

In [ ]:
# A function to fit the second order polynomial
def fit_poly(img_shape, leftx, lefty, rightx, righty):
     # Fit a second order polynomial to each with np.polyfit()
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    # Calc both polynomials using ploty, left_fit and right_fit
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

# A function to search around the polynomial 
def search_around_poly(binary_warped):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Set the area of search based on activated x-values
    # within the +/- margin of our polynomial function
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result

In [ ]:
# Polynomial fit values from the previous frame
# Grab the actual values from the previous step
# Will be optimized later
left_fit = np.array([-5.82876789e-05, 9.89023579e-03, 3.63726419e+02])
right_fit = np.array([1.13367624e-04, -3.17551122e-01, 1.14925867e+03])

# Apply the search pipepline to binary image
boundary = search_around_poly(color_img_warped)

# View output image
plt.imshow(boundary)

### Another Sliding Window Search

In [ ]:
# A function to create the mask window
def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

# A function to find window centroids
def find_window_centroids(image, window_width, window_height, margin):
    
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(image[int(3*image.shape[0]/4):,:int(image.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(image[int(3*image.shape[0]/4):,int(image.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(image.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(image.shape[0]/window_height)):
	    # convolve the window into the vertical slice of the image
	    image_layer = np.sum(image[int(image.shape[0]-(level+1)*window_height):int(image.shape[0]-level*window_height),:], axis=0)
	    conv_signal = np.convolve(window, image_layer)
	    # Find the best left centroid by using past left center as a reference
	    # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
	    offset = window_width/2
	    l_min_index = int(max(l_center+offset-margin,0))
	    l_max_index = int(min(l_center+offset+margin,image.shape[1]))
	    l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
	    # Find the best right centroid by using past right center as a reference
	    r_min_index = int(max(r_center+offset-margin,0))
	    r_max_index = int(min(r_center+offset+margin,image.shape[1]))
	    r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
	    # Add what we found for that layer
	    window_centroids.append((l_center,r_center))

    return window_centroids

# A function to search sliding window by using convolution
def convolution_search(img):
    # Read in a thresholded image
    warped = img
    # window settings
    window_width = 50 
    window_height = 80 # Break image into 9 vertical layers since image height is 720
    margin = 100 # How much to slide left and right for searching

    window_centroids = find_window_centroids(warped, window_width, window_height, margin)

    # If we found any window centers
    if len(window_centroids) > 0:

        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)

        # Go through each level and draw the windows 	
        for level in range(0,len(window_centroids)):
            # Window_mask is a function to draw window areas
            l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
            r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
            # Add graphic points from window mask here to total pixels found 
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

        # Draw the results
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        zero_channel = np.zeros_like(template) # create a zero color channel
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
        warpage= np.dstack((warped, warped, warped))*255 # making the original road pixels 3 color channels
        output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
 
    # If no window centers found, just display orginal road image
    else:
        output = np.array(cv2.merge((warped,warped,warped)),np.uint8)
    
    return output

In [ ]:
# Apply the search pipepline to binary image
boundary_window = convolution_search(color_img_warped)

# View output image
plt.imshow(boundary_window)

# 7) Warp the detected lane boundaries back onto the original image.

In [ ]:
# A helper function to combine the generated binary image with initial image
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img`: output image of the process    
    `initial_img`: the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

### Lane boundaries detected by sliding window & search from prior

In [ ]:
# Unwarped the binary image
# Apply perspective transform reversely by switching src & dst
unwarped, M_perspective_reverse = warped(boundary, dst, src)

# Combine unwarped binary image with boundaries to initial image
combined = weighted_img(test_img_undist, unwarped)

# Show unwarped & combined image
show2img(unwarped,combined, "Unwarped image with boundaries", "Combined with original image")

### Lane boundaries detected by convolution search

In [ ]:
# Unwarped the binary image
# Apply perspective transform reversely by switching src & dst
unwarped, M_perspective_reverse = warped(boundary_window, dst, src)

# Combine unwarped binary image with boundaries to initial image
combined = weighted_img(test_img_undist, unwarped)

# Show unwarped & combined image
show2img(unwarped,combined, "Unwarped image with boundaries", "Combined with original image")

# 8) Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

# Pipeline to process with video

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
# Show targeted videos in a line (for easy confirmation)
HTML("""<video width="320" height="180" title="Project video" controls> 
<source src="project_video.mp4">
</video>
<video width="320" height="180" title="Challenge video" controls> 
<source src="challenge_video.mp4">
</video>
<video width="320" height="180" title="Harder challenge video" controls> 
<source src="harder_challenge_video.mp4">
</video>""")

In [ ]:
# Pipepline to process each image
def process_image(img):
    # Undistort the given image
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    # Create binary image by applying thresholds
    # Temporarily use color threshold method
    binary = color_thresh(undist, color_thresh=(200, 255))
    
    # Create a warped image by perspective transform
    binary_warped, M_perspective = warped(binary, src, dst)
    
    """
    # Find lane pixels by raw lane boundaries
    left_fit = np.array([-5.82876789e-05, 9.89023579e-03, 3.63726419e+02])
    right_fit = np.array([1.13367624e-04, -3.17551122e-01, 1.14925867e+03])
    boundary = search_around_poly(binary_warped)
    """
    # Apply the search pipepline to binary image
    boundary = convolution_search(binary_warped)
    
    # Unwarped the binary image
    # Apply perspective transform reversely by switching src & dst
    binary_unwarped, M_perspective_reverse = warped(boundary, dst, src)
    
    # Warp the detected lane boundaries back onto the original image
    result = weighted_img(undist,binary_unwarped)
    
    return result

In [ ]:
# Apply the pipeline to process project video
# Firstly try with the first 10s
project_video = VideoFileClip("project_video.mp4").subclip(0,10)
# Then try with full video
#project_video = VideoFileClip("project_video.mp4")
project_video_output = 'output_videos/project_video_output.mp4'
project_video_process = project_video.fl_image(process_image)
%time project_video_process.write_videofile(project_video_output, audio=False)

In [ ]:
# Show 2 videos (before and after) in a line for easy comparison
HTML("""<video width="480" height="270" title="Project video" controls> 
<source src="project_video.mp4">
</video>
<video width="480" height="270" title="Output video" controls> 
<source src="output_videos/project_video_output.mp4">
</video>""")

## Apply the pipeline to process with challenge videos to find the shortcomings

In [ ]:
# Apply the pipeline to challenge video
# Firstly try with the first 10s
challenge_video = VideoFileClip("challenge_video.mp4").subclip(0,10)
# Then try with full video
#challenge_video = VideoFileClip("challenge_video.mp4")
challenge_video_output = 'output_videos/challenge_video_output.mp4'
challenge_video_process = challenge_video.fl_image(process_image)
%time challenge_video_process.write_videofile(challenge_video_output, audio=False)

In [ ]:
# Show 2 videos (before and after) in a line for easy comparison
HTML("""<video width="480" height="270" title="Challenge video" controls> 
<source src="challenge_video.mp4">
</video>
<video width="480" height="270" title="Output video" controls> 
<source src="output_videos/challenge_video_output.mp4">
</video>""")

In [ ]:
# Apply the pipeline to challenge video
# Firstly try with the first 10s
harder_challenge_video = VideoFileClip("harder_challenge_video.mp4").subclip(0,10)
# Then try with full video
#harder_challenge_video = VideoFileClip("harder_challenge_video.mp4")
harder_challenge_video_output = 'output_videos/harder_challenge_video_output.mp4'
harder_challenge_video_process = harder_challenge_video.fl_image(process_image)
%time harder_challenge_video_process.write_videofile(harder_challenge_video_output, audio=False)

In [ ]:
# Show 2 videos (before and after) in a line for easy comparison
HTML("""<video width="480" height="270" title="Harder challenge video" controls> 
<source src="harder_challenge_video.mp4">
</video>
<video width="480" height="270" title="Output video" controls> 
<source src="output_videos/harder_challenge_video_output.mp4">
</video>""")